In [1]:
from abc import ABC, abstractmethod
from datetime import datetime, timedelta, date
import os

import housekeeper

class DataManager(ABC):
    
    def __init__(self):
        
        self.__myHousekeeper = housekeeper.instance_class()
        self.__config_filename = "tickers_config.json"
        self.__dir_list = ['Data', 'Tickers', 'Dummy1']
        self.__upper_stages = 0
        self.__tickers_config_list = []
        self.__tickers_list = []
        self.__active_tickers_list = []
        self.__selected_tickers_list = []
        self.__timestamp = ''
        self.__markets = []
        self.__last_date_flag = False
    

    def get_config_filename(self):
        return self.__config_filename
    
    def set_config_filename(self, config_filename):
        self.__config_filename = config_filename
        
    def get_dir_list(self):
        return self.__dir_list
    
    def set_dir_list(self, dir_list):
        self.__dir_list = dir_list
    
    def get_upper_stages(self):
        return self.__upper_stages
    
    def set_upper_stages(self, upper_stages):
        self.__upper_stages = dir_list
        
    def get_last_date_flag(self):
        return self.__last_date_flag
    
    def set_last_date_flag(self, last_date_flag):
        self.__last_date_flag = last_date_flag
        
    def get_tickers_config(self):
        return self.__tickers_config_list
    
    def set_tickers_config(self, tickers_config_list):
        self.__tickers_config_list = tickers_config_list
    
    def get_tickers(self):
        return self.__tickers_list
    
    def set_tickers(self, tickers_list):
        self.__tickers_list = tickers_list
        
    def get_active_tickers(self):
        return self.__active_tickers_list
    
    def set_active_tickers(self, active_tickers_list):
        self.__active_tickers_list = active_tickers_list
        
    def get_selected_tickers(self):
        return self.__selected_tickers_list
    
    def set_selected_tickers(self, selected_tickers_list):
        self.__selected_tickers_list = selected_tickers_list
    
    def get_timestamp(self):
        return self.__timestamp
    
    def set_timestamp(self, timestamp):
        self.__timestamp = timestamp
    
    def get_markets(self):
        return self.__markets
    
    def set_markets(self, markets):
        self.__markets = markets
    
    def load_tickers_config(self):
        data = self.__myHousekeeper.load_json_to_list(self.__dir_list, self.__config_filename)
        self.set_tickers_config(data)
        
    def save_tickers_config(self):
        #No invocar a esta función sin previamente haber cargado tickers_config. O se sobreescribe tickers_config
        tickers_config = self.get_tickers_config()
        self.__myHousekeeper.list_dict_to_json(self.get_dir_list(), 
                                               self.get_upper_stages(), 
                                               self.get_config_filename(), 
                                               self.get_tickers_config())
    
    def initialize_metadata(self):
        self.load_tickers_config()
        data = self.get_tickers_config()
        self.set_timestamp(data['metadata'][0]['timestamp'])
        self.set_tickers(data['data'])
        
    def initialize_config_tickers(self):
        # Get markets, get active_tickers
        markets = []
        active_tickers_ = []
        self.initialize_metadata()
        data = self.get_tickers()
        for d in data:
            markets.append(d['market'])
            if d['active_type']=='stock' and d['active_flag']:
                active_tickers_.append(d)
            elif d['active_type']=='ETF':
                active_tickers_.append(d)
        self.set_active_tickers(active_tickers_)
        self.set_markets(list(set(markets)))
    
    def api_selected_tickers(self):
        #Se recarga el tickers_config para info actualizada de los tickers.
        self.initialize_config_tickers()
        # Se despliegan los tickers activos en la UI para que el usuario elija qué tickers quiere actualizar el data.
        ticker_list = self.get_tickers()
        self.set_selected_tickers(ticker_list[0:3])
        
        #return self.get_active_tickers() #TODO
    
    def update_timeseries_download_date(self, selected_tickers_to_update):
        config_ticker_list = self.get_tickers_config()
        today = date.today()
        # LAs fechas se guardan en formato %m-%d-%Y
        [t.update({'data_update':today.strftime("%m-%d-%Y")}) for t in config_ticker_list['data'] if t in selected_tickers_to_update]
        self.set_tickers_config(config_ticker_list)
        self.save_tickers_config()
         
    def load_ticker_data(self, file_name):
        return self.__myHousekeeper.csv_to_df(self.__dir_list,
                                              file_name)
    
    def save_ticker_data(self, file_name, data):
        self.__myHousekeeper.df_to_csv(self.__dir_list,
                                       self.__upper_stages, file_name, data)
        
    

In [2]:
import yfinance as yf
import pandas as pd

class DataManager_YahooFinance(DataManager):
    
    def __init__(self):
        super().__init__()
        
    
    def download_ticker_data_from_scratch(self, ticker, ticker_key):
        print('Downloading from scratch historic data of: ' + ticker)
        data_csv = yf.download(ticker)
        data_csv.insert(loc=0, column='Date', value=pd.to_datetime(data_csv.index, errors='coerce'))
        data_csv['Date'] = [time.date() for time in data_csv['Date']]
        data_csv.reset_index(drop=True, inplace=True)
        self.save_ticker_data(ticker_key,data_csv )
        return data_csv
    
    def download_ticker_data_from_last_date(self, ticker, ticker_key, start_date):
        print('Updating historic data of: ' + ticker)
        # 1. Descargar datos desde la ultima fecha
        data_csv = yf.download(ticker, start = start_date)
        data_csv.insert(loc=0, column='Date', value=pd.to_datetime(data_csv.index, errors='coerce'))
        data_csv['Date'] = [time.date() for time in data_csv['Date']]
        print('Downloaded(sessions)', len(data_csv))
        # 2. Cargar el csv
        data_csv_local = DM_YF.load_ticker_data(ticker_key)
        # 3. Apendear los datos que faltan, resetear el index y esta será la nueva varaible data_csv
        data_csv = pd.concat([data_csv_local, data_csv], ignore_index = True)
        data_csv.reset_index(drop=True, inplace=True)
        data_csv.drop(data_csv.columns[0], axis = 1, inplace = True)
        # 4. Guardar los datos sobreescribiendo el archivo anterior
        self.save_ticker_data(ticker_key, data_csv)
        #return data_csv
    
    def last_date_download(self, ticker_dict):
        # Local variables
        last_date_str_ = ticker_dict['data_update']
        ticker_key_ = ticker_dict['tickerKey']
        ticker = ticker_dict['feeds']['ticker']
        # 3 casos: A) last_date is None -> from scratch, B) last >= today -> no hay descarga C) start < today (else) -> download_ticker_data_from_last_date
        if last_date_str_ is None: # Aquí va un download_from_scratch
            print(ticker + " is not found in database, adding ----")
            #data_csv = yf.download(ticker) # Aquí va un download_from_scratch
            self.download_ticker_data_from_scratch(ticker, ticker_key_)
            return
        now = datetime.now()
        last_date = datetime.strptime(last_date_str_, '%m-%d-%Y')
        delta = now - last_date
        start_date = last_date + timedelta(days=+1)
        if delta.days <= 0: # Aquí no hay download
            print('Data of ', ticker_key_ ,'is already updated')
            return
        else: # Función download_ticker_data_from_last_date
            self.download_ticker_data_from_last_date(ticker, ticker_key_, start_date)
            delta = now - start_date
            print('Downloaded(days): ', delta.days)
            #return data_csv
    
    
    def timeseries_download_manager(self, ticker_dict):
        if self.get_last_date_flag(): # From last date
            print('Download ', ticker_dict['tickerKey'],' from last updated_date')
            self.last_date_download(ticker_dict)
        else: # From scratch
            print('Download', ticker_dict['tickerKey'],' from scratch')
            self.download_ticker_data_from_scratch(ticker_dict['feeds']['ticker'],ticker_dict['tickerKey'])
        
    
    def download_selected_tickers(self):
        # Se almacenan los tickers que van a se actualizados y se guarda la fecha de actualización en el ticker_config. 
        # 1.- Almacenar selected_Tickers from user selection and a default option.
        #selected_tickers_list = self.api_active_tickers()
        self.api_selected_tickers()
        #2.- Establecer el tipo de descarga: last_date(True) / from scratch(False, default) 
        self.set_last_date_flag(False)
        #3.- Descargar los selected_tickers. Enganchar con timeseries_download_manager
        [self.timeseries_download_manager(t) for t in self.get_selected_tickers()]
        # 4.- Actualizar data_update en tickers_config de los tickers descargados
        self.update_timeseries_download_date(self.get_selected_tickers())
    
    
    def download_market_data(self, markets, _last_date_flag): #TODO: especificar el subconjunto en selected tickers. Para que se actualice la fecha data_update
        print('Download market ticker')
        #1.- Almacenar en selected_ticker los tickers correspondientes a un market
        #Se recarga el tickers_config para info actualizada de los tickers.
        self.initialize_config_tickers()
        # Se despliegan los tickers activos en la UI para que el usuario elija qué tickers quiere actualizar el data.
        active_ticker_list = self.get_active_tickers()
        ticker_list = [t for t in active_ticker_list if t['market'] in markets]
        self.set_selected_tickers(ticker_list)
        #2.- Establecer el tipo de descarga: last_date(True) / from scratch(False, default) 
        self.set_last_date_flag(_last_date_flag)
        #3.- Descargar los selected_tickers. Enganchar con timeseries_download_manager
        #tickers = self.get_active_tickers()
        #[DM_YF.download_ticker_data_from_scratch(t['feeds']['ticker'], t['tickerKey']) for t in tickers if t['market'] in markets]
        [self.timeseries_download_manager(t) for t in self.get_selected_tickers()]
        # 4.- Actualizar data_update en tickers_config de los tickers descargados
        self.update_timeseries_download_date(self.get_selected_tickers())
        
    def download_all_markets(self):
        print('Download ALL MARKETS')
        self.download_market_data(self.get_markets())
    

In [3]:
DM_YF = DataManager_YahooFinance()
DM_YF.load_tickers_config() # Cargar archivo "tickers_config.json"
data = DM_YF.get_tickers_config() # Obtener variable __tickers_config_list
#DM_YF.initialize_metadata()
DM_YF.initialize_config_tickers() # initialize_metadata(timestamp and set tickers_list) and set __active_tickers_list 
DM_YF.get_timestamp(), DM_YF.get_markets()
#print(DM_YF.start_date_calculation("03-24-2021"))

nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/tickers_config.json
nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/tickers_config.json


('30/03/2021 20:01:20', ['SP500', 'IBEX35', 'DAX30', 'CAC40'])

In [4]:
DM_YF.download_selected_tickers()

nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/tickers_config.json
Download ANA.MC.TT  from scratch
[*********************100%***********************]  1 of 1 completed
nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/ANA.MC.TT.csv
Download ACX.MC.TT  from scratch
[*********************100%***********************]  1 of 1 completed
nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/ACX.MC.TT.csv
Download ACS.MC.TT  from scratch
[*********************100%***********************]  1 of 1 completed
nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/ACS.MC.TT.csv
Root dir:  C:\Users\alvaro\Repos\Python_knowledge\Appdatabase project
parent_dir_path:  C:\Users\alvaro\Repos\Python_knowledge\Appdatabase project
dir_list:  ['Data', 'Tickers', 'Dummy1']
nested_dir_path:  /Data/Tickers/Dummy1
file path:  C:\Users\alvaro\Repos\Python_knowledge\Appdatabase project/Data/Tickers/Dummy1/tickers_config.json


In [4]:
DM_YF.download_market_data('IBEX35', False)

Download market ticker
nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/tickers_config.json
Download ANA.MC.TT  from scratch
[*********************100%***********************]  1 of 1 completed
nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/ANA.MC.TT.csv
Download ACX.MC.TT  from scratch
[*********************100%***********************]  1 of 1 completed
nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/ACX.MC.TT.csv
Download ACS.MC.TT  from scratch
[*********************100%***********************]  1 of 1 completed
nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/ACS.MC.TT.csv
Download AENA.MC.TT  from scratch
[*********************100%***********************]  1 of 1 completed
nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/AENA.MC.TT.csv
Download ALM.MC.TT  from scratch
[*********************100%***********************]  1 of 1 completed
nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/ALM.MC.TT.csv
Download 

In [ ]:
tickers = DM_YF.get_tickers()
tickers[3]['data_update']
#print(tickers[3]['data_update'])

In [ ]:
#DM_YF.timeBounded_download('ACX.MC', 'ACX.MC.TT',"03-23-2021")
DM_YF.timeBounded_download(tickers[4]['ticker'], tickers[4]['tickerKey'],tickers[4]['data_update'])


In [ ]:
DM_YF.download_ticker_data_from_last_date('ACX.MC', 'ACX.MC.TT',"2021-03-24" )

In [ ]:
def start_date_definition(self):
    return start_date
    

In [ ]:
last_date_str = a[0]['data_update']
last_date = datetime.strptime(last_date_str, '%m-%d-%Y')
print(last_date)
start_date = last_date + timedelta(days=+1)
print(start_date)
print(start_date >= datetime.now())
#start_date = None
ticker = 'ANA.MC'
def timeBounded_download(self, ticker, ticker_key, last_date):
    # Se activa opción de descargar desde la última fecha 
    # 3 casos: A) start is None, B) start >= today C) start < today
    delta = now-start_date
    if start_date is None: # Aquí va un download_from_scratch
        print(ticker + " is not found in database, adding ----")
        #data_csv = yf.download(ticker) # Aquí va un download_from_scratch
        self.download_ticker_data_from_scratch(ticker, ticker_key)
    elif delta.days <= 0: # Aquí no hay download
        print('Data is already updated')
        return
    else: # Aquí un download_from_last_date
        #data_csv = yf.download(ticker, start = start_date)
        # Función para calcular la start_date
        start_date = self.start_date_calculation(last_date)
        # Función download_ticker_data_from_last_date
        self.download_ticker_data_from_last_date(ticker, ticker_key, start_date)
        delta = now-start_date
        print('Downloaded(days): ', delta.days)
    return data_csv
    
    #return data_csv = yf.download(ticker)

In [ ]:
data_csv = timeBounded_download('ACS')
data_csv.head()

In [ ]:
last_date_str = a[0]['data_update']
print(type(last_date))
#last_date = pd.to_datetime(last_date, errors='coerce')
last_date = datetime.strptime(last_date_str, '%m-%d-%Y')
#print(last_date)
start_date = last_date + timedelta(days=-2)
#start_date = pd.to_datetime(start_date, errors='coerce')
#start_date = None
print(last_date)
print(start_date), print(type(start_date))
today = date.today()
now = datetime.now()
#print(today), print(type(today))
print(now), print(type(now))
#print(today.strftime("%d-%m-%Y"))
print(start_date >= now)
delta = start_date - now
print(delta.days)
#data_csv = yf.download('ANA.MC', start = start_date)
#data_csv.head()

In [ ]:
a = DM_YF.get_active_tickers()
a[0]['data_update'], a[2]['data_update']

In [ ]:
#data_csv = yf.download('ANA.MC')
last_date = a[0]['data_update']
last_date = pd.to_datetime(last_date, errors='coerce')
last_date.date()
start_date = last_date + timedelta(days=1)
start_date_1 = last_date + timedelta(days=-2)
last_date, start_date, start_date_1

In [ ]:
data_csv = yf.download('ANA.MC', start=start_date_1, end=datetime.now())
#data_csv = yf.download('ANA.MC')
data_csv.head(-1)

In [ ]:
last_date = a[0]['data_update'] 
last_date = pd.to_datetime(last_date, errors='coerce')
last_date = last_date + timedelta(days=-2)
last_date

In [ ]:
#start = last_date
start = None
end = end=datetime.now()
def fun1(start = start, end = end):
    if start is not None:
        data_csv = yf.download('ANA.MC',start,  end)
    else:
        data_csv = yf.download('ANA.MC')
    return data_csv

In [ ]:
data_csv = fun1()
type(data_csv)
data_csv.head(-1)

In [ ]:
date_dict = {'start':last_date}
def fun2(**kwargs):
    data_csv = yf.download('ANA.MC', **kwargs)
        #data_csv = yf.download('ANA.MC',start,  end)
    return data_csv

In [ ]:
data_csv = fun2()
type(data_csv)
data_csv.head(-1)

In [ ]:
date_dict = {'start':last_date}
data_csv = yf.download('ANA.MC', **date_dict)
data_csv.head()

In [ ]:
data_csv = yf.download('ANA.MC', end=datetime.now())
#data_csv = yf.download('ANA.MC')
data_csv.head(-1)

### Arquitectura descarga tickers
fdsa
fdas

In [ ]:
def f1(a = 1):
    print(a)

In [ ]:
f1(424)

In [ ]:
tickers[0]

In [ ]:
tickers[0]['feeds']['ticker']

In [ ]:
data_ticker = DM_YF.download_ticker_data(tickers[0]['feeds']['ticker'], tickers[0]['tickerKey'])
#data_ticker

In [ ]:
data_ticker.head(), type(data_ticker)

In [ ]:
#DM_YF.download_market_data('IBEX35')

In [ ]:
markets = DM_YF.get_markets()
markets

In [ ]:
DM_YF.download_all_markets()

In [ ]:
def Get_Last_Data(ticker, market, backup):
    # Check if the market is in the database. In case that not exist, WARNING and return empty csv.
    markets = os.listdir('./Data/')
    if market not in markets:
        print("There is no market in database, please add new market and fill with data")
        data_csv = []
        return data_csv

    # If market exists between the markets loaded, then go to market folder and load shares list
    # Get shares of a market from the folder that it is stored
    shares = os.listdir('./Data/' + market)
    shares = [s.replace('.csv', '') for s in shares]

    # Function to download full historic data, used for backup and in firts time download
    def dowload_historic_data(ticker):
        print('Downloading historic data of: ' + ticker)
        data_csv = yf.download(ticker)
        data_csv.insert(loc=0, column='Date', value=pd.to_datetime(data_csv.index, errors='coerce'))
        data_csv['Date'] = [time.date() for time in data_csv['Date']]
        data_csv.reset_index(drop=True, inplace=True)
        return data_csv

    if ticker in shares:  # IN CASE THAT THE SHARE EXISTS PREVIOUSLY. If share in LISTA_ACCIONES
        if backup:  # BACK UP CASE. Download data from the historic in YahooFinance
            data_csv = dowload_historic_data(ticker)
        else:  # NO BACK UP CASE. Download data from the last date updated
            # Load de data
            print("-------------------------------------------")
            print(ticker + " exists in Database. " + "Opening " + ticker)
            data_csv = pd.read_csv("./Data/" + market + "/" + ticker + ".csv")
            # Read and get the date of the last day
            last_date = data_csv["Date"].iloc[-1]
            last_date = pd.to_datetime(last_date, errors='coerce')
            last_date.date()
            # If last date is today, then return the file with no modifications
            if last_date >= date.today():
                print("File is already updated. No modifications.")
                # print ("-------------------------------------------")
                return data_csv
            # print("Updating " + ticker +' from '+ last_date +' until '+date.today()+'(today)' )
            print('Updating ' + ticker + ' until today')
            start_date = last_date + timedelta(days=1)
            # Download data from the selected date
            data = yf.download(ticker, start=start_date, end=datetime.now())
            data.insert(loc=0, column='Date', value=pd.to_datetime(data.index, errors='coerce'))
            data['Date'] = [time.date() for time in data['Date']]
            data.reset_index(drop=True, inplace=True)
            data_csv = data_csv.append(data, ignore_index=True)
    else:  # IN CASE THAT THE SHARE DOESNT EXISTS PREVIOUSLY. Download historic data.
        print(ticker + " is not found in database, adding ----")
        data_csv = dowload_historic_data(ticker)
    return data_csv

### Development layer 

In [ ]:
def download_ticker_data_from_last_date(ticker, ticker_key, start_date):
        print('Updating historic data of: ' + ticker)
        # 1. Descargar datos desde la ultima fecha
        data_csv = yf.download(ticker, start = start_date)
        data_csv.insert(loc=0, column='Date', value=pd.to_datetime(data_csv.index, errors='coerce'))
        data_csv['Date'] = [time.date() for time in data_csv['Date']]
        # 2. Cargar el csv
        #data_csv_local = DM_YF.load_ticker_data(ticker_key)
        data_csv_local = pd.read_csv('./Data/Tickers/Dummy1/ACX.MC.TT.csv')
        # 3. Apendear los datos que faltan, resetear el index y esta será la nueva varaible data_csv
        print(type(data_csv_local)),print(type(data_csv))
        data_csv = pd.concat([data_csv_local, data_csv], ignore_index = True)
        data_csv.reset_index(drop=True, inplace=True)
        data_csv.drop(data_csv.columns[0], axis = 1, inplace = True)
        #data_csv.drop(1)
        # 4. Guardar los datos
        #self.save_ticker_data(ticker_key,data_csv )
        return data_csv

In [ ]:
download_ticker_data_from_last_date('ANA.MC', 'ANA.MC.TT',"2021-03-23")

In [ ]:
a = DM_YF.get_tickers()
a[0]['data_update'], a[9]['data_update']

In [ ]:
last_date = a[0]['data_update']
print(type(last_date))
#last_date = pd.to_datetime(last_date, errors='coerce')
last_date = datetime.strptime(last_date, '%m-%d-%Y')

In [ ]:
data = DM_YF.load_ticker_data('ACX.MC.TT')
data

In [ ]:
import pandas as pd
data = pd.read_csv('./Data/Tickers/Dummy1/ACX.MC.TT.csv')
data

In [ ]:
def download_market_data(self, market):
    print('Download market ticker')
    tickers = self.get_active_tickers()
    [DM_YF.download_ticker_data(t['feeds']['ticker'], t['tickerKey']) for t in tickers if t['market'] in markets]

In [ ]:
tickers = DM_YF.get_active_tickers() # Get active tickers
#tickers

In [ ]:
markets = ['IBEX35']
[print(t) for t in tickers if t['market'] in markets]

DM_YF.download_ticker_data(tickers[0]['feeds']['ticker'], tickers[0]['tickerKey'])

In [ ]:
[DM_YF.download_ticker_data(t['feeds']['ticker'], t['tickerKey']) for t in tickers if t['market'] in markets]

In [ ]:
import housekeeper

In [ ]:
dir_list = ['Data']
file_name = 'tickers_config.json'

In [ ]:
myHousekeeper = housekeeper.instance_class() 
data = myHousekeeper.load_json_to_list(dir_list, file_name)
#data

In [ ]:
markets = []
active_tickers = []
for d in data:
    markets.append(d['market'])
    if d['active_type']=='stock' and d['active_flag']:
        active_tickers.append(d)
    elif d['active_type']=='ETF':
        active_tickers.append(d)
markets = set(markets)    

In [ ]:
markets, active_tickers

In [ ]:
data[0:2]['market']

In [ ]:
data[0:2]

In [ ]:
keyValList = ['a']
data[0]['market'] in keyValList

In [ ]:
#data

In [ ]:
if data[0]['active_flag']:
    print('hola')

In [ ]:
markets

In [ ]:
'''
>>> exampleSet = [{'type':'type1'},{'type':'type2'},{'type':'type2'}, {'type':'type3'}]
>>> keyValList = ['type2','type3']
>>> expectedResult = [d for d in exampleSet if d['type'] in keyValList]
>>> expectedResult
[{'type': 'type2'}, {'type': 'type2'}, {'type': 'type3'}]
'''

In [ ]:
'''
>>> list(filter(lambda d: d['type'] in keyValList, exampleSet))
[{'type': 'type2'}, {'type': 'type2'}, {'type': 'type3'}]
'''

https://blog.finxter.com/how-to-filter-a-list-of-dictionaries-in-python/#Where_to_Go_From_Here

In [ ]:
data = DM_YF.get_tickers_config()

In [ ]:
DM_YF.load_tickers_config()

In [ ]:
data[1]

In [ ]:
list = ['a', 'b', 'c']

In [ ]:
list.pop(0)
list

In [ ]:
a = list.pop(0)
a